In [3]:
!pip install tensorflow matplotlib pandas

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import files
from PIL import Image
import io

In [13]:
def load_data(file):
    df = pd.read_csv(file)
    labels = df['label'].values
    images = df.drop('label', axis=1).values.reshape(-1, 28, 28, 1)
    images = images / 255.0
    labels = to_categorical(labels)
    return images, labels
x_train, y_train = load_data('sign_mnist_train.csv')
x_test, y_test = load_data('sign_mnist_test.csv')

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [22]:
model = Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')
])


In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.2747 - loss: 2.3982 - val_accuracy: 0.8977 - val_loss: 0.4031
Epoch 2/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.7868 - loss: 0.6235 - val_accuracy: 0.9663 - val_loss: 0.1494
Epoch 3/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 40s 29ms/step - accuracy: 0.8709 - loss: 0.3617 - val_accuracy: 0.9954 - val_loss: 0.0547
Epoch 4/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.9218 - loss: 0.2192 - val_accuracy: 0.9965 - val_loss: 0.0302
Epoch 5/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.9415 - loss: 0.1706 - val_accuracy: 0.9998 - val_loss: 0.0102
Epoch 6/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.9501 - loss: 0.1404 - val_accuracy: 0.9998 - val_loss: 0.0058
Epoch 7/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.9572 - loss: 0.1151 - val_accuracy: 1.0000 - val_loss: 0.0047
Epoch 8/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.9643 - loss: 0.1016 - 

In [25]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9223 - loss: 0.3211
Test Accuracy: 0.9211


In [28]:
model.save('/content/sign_language_mnist_model.keras')

In [30]:
tf_model = load_model('/content/sign_language_mnist_model.keras', compile=False)
tf_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
def predict_uploaded_image():
    label_map = {
        0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I',
        9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P', 15: 'Q', 16: 'R',
        17: 'S', 18: 'T', 19: 'U', 20: 'V', 21: 'W', 22: 'X', 23: 'Y'
    }
    print("Upload a 28x28 grayscale image for prediction")
    uploaded_img = files.upload()
    for filename in uploaded_img:
        img = Image.open(io.BytesIO(uploaded_img[filename])).convert('L').resize((28, 28))
        img_array = np.array(img).reshape(1, 28, 28, 1) / 255.0
        prediction = tf_model.predict(img_array)
        predicted_label = np.argmax(prediction)
        predicted_letter = label_map.get(predicted_label, '?')
        print(f"Predicted Label: {predicted_label} => Letter: {predicted_letter}")

In [36]:
predict_uploaded_image()

Upload a 28x28 grayscale image for prediction


Saving a-sign-language-4561deac-cfdedd5a-1920w.PNG to a-sign-language-4561deac-cfdedd5a-1920w (2).PNG
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Label: 0 => Letter: A
